# Multi i-TED characterization
## Second: Checking hardware fix for iTED-B-4

- In the [previous notebook](./PyROOT_resolution_Cs137v1.html) the absorver 4 of iTED-B had degradaded considerably in energy resolution.
- From a lab book, this crystal was identified has having had problems due to poor light shielding that resulted in a pixel firing constantly.
- Black gum was placed around the aluminium casing in order to better shield the crystal from light.
- A new measurement was performed with the fix and the results are analysed below.

In [1]:
pkg_ver = lambda pkg: "{:<20}{:}".format(pkg.__name__,pkg.__version__)

# ROOT
import uproot
print(pkg_ver(uproot))
import ROOT

# Machine Learning
import sklearn
print(pkg_ver(sklearn))
import torch
print(pkg_ver(torch))

# Data science
import scipy
print(pkg_ver(scipy))
import numpy
print(pkg_ver(numpy))
import pandas
print(pkg_ver(pandas))

# Visualizations
import matplotlib
print(pkg_ver(matplotlib))
import matplotlib.pyplot as plt

import tqdm
print(pkg_ver(tqdm))

uproot              4.3.5
Welcome to JupyROOT 6.26/10
sklearn             1.2.0
torch               2.0.0rc5
scipy               1.10.1
numpy               1.24.2
pandas              1.5.3
matplotlib          3.6.3
tqdm                4.62.3


In [2]:
%jsroot on

In [3]:
class spectrum:
    
    def __init__(self, TH1D_, iTED_, Crystal_, Configuration_, Window_, Calibration_):  

        self.__TH1D = TH1D_
        self.__iTED = iTED_
        self.__Crystal = Crystal_
        self.__Configuration = Configuration_
        self.__Window = Window_
        self.__Calibration = Calibration_
        
    def __call__(self, ch):
        return numpy.polyval(self.__Calibration[::-1],ch)
    
    def TH1D(self):
        return self.__TH1D
    
    def Calibration(self):
        return self.__Calibration
    
    def iTED(self):
        return self.__iTED
    
    def Crystal(self):
        return self.__Crystal
    
    def Configuration(self):
        return self.__Configuration
    
    def Window(self):
        return self.__Window
    
    def __repr__(self):
        return "iTED: {},Crystal: {},Configuration: {},Window: {}".format(
            self.__iTED,
            self.__Crystal,
            self.__Configuration,
            self.__Window
        )
    
    def __str__(self):
        return "{}.{}.{}.{}".format(
            self.__iTED,
            self.__Crystal,
            self.__Configuration,
            self.__Window
        )

In [4]:
iTEDA_cal = pandas.read_csv(
    "../../data/nTOF_March2022/Energy_Calibrations_02_03_2022/Energy_Calibrations_iTEDA.dat",
    delim_whitespace=True,
    names=["P0","P1","P2"]
)

iTEDB_cal = pandas.read_csv(
    "../../data/nTOF_March2022/Energy_Calibrations_02_03_2022/Energy_Calibrations_iTEDB.dat",
    delim_whitespace=True,
    names=["P0","P1","P2"]
)

iTEDC_cal = pandas.read_csv(
    "../../data/nTOF_March2022/Energy_Calibrations_02_03_2022/Energy_Calibrations_iTEDC.dat",
    delim_whitespace=True,
    names=["P0","P1","P2"]
)

iTEDD_cal = pandas.read_csv(
    "../../data/nTOF_March2022/Energy_Calibrations_02_03_2022/Energy_Calibrations_iTEDD.dat",
    delim_whitespace=True,
    names=["P0","P1","P2"]
)

iTED_cal = pandas.concat([iTEDA_cal.T, iTEDB_cal.T, iTEDC_cal.T, iTEDD_cal.T], axis=1, keys=['A', 'B', 'C', 'D'])

In [5]:
config = [885, 888, 8811]
CW = [100,150,200,250]

ited = ["A","B","C","D"]
crystal = [0,1,2,3,4]

midx = pandas.MultiIndex.from_product(
    [config, CW],
    names=['Configuration', 'Window']
)

mcol = pandas.MultiIndex.from_product(
    [ited, crystal],
    names=['iTED', 'Crystal']
) 

spectra = pandas.DataFrame(index = midx, columns = mcol)

# Access using spectra[iTED,Crystal][Configuration,Window]

In [6]:
iTEDA = {
    "8811-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDA-A2_D.2023_02_23_T.12_21_13_C.itedABCD_lab_2023.02.22_4.0v_8811_300s_CW100.root","READ"),
    "8811-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDA-A2_D.2023_02_23_T.12_21_13_C.itedABCD_lab_2023.02.22_4.0v_8811_300s_CW150.root","READ"),
    "8811-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDA-A2_D.2023_02_23_T.12_21_13_C.itedABCD_lab_2023.02.22_4.0v_8811_300s_CW200.root","READ"),
    "8811-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDA-A2_D.2023_02_23_T.12_21_13_C.itedABCD_lab_2023.02.22_4.0v_8811_300s_CW250.root","READ"),
    "888-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDA-A2_D.2023_02_23_T.13_40_46_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100.root","READ"),
    "888-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDA-A2_D.2023_02_23_T.13_40_46_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW150.root","READ"),
    "888-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDA-A2_D.2023_02_23_T.13_40_46_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW200.root","READ"),
    "888-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDA-A2_D.2023_02_23_T.13_40_46_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW250.root","READ")
}

iTEDB = {
    "8811-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_AfterReconnecting_gum_iTEDB-middle_D.2023_03_02_T.15_10_36_C.itedABCD_lab_2023.02.22_4.0v_8811_300s_CW100.root","READ"),
    "8811-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_AfterReconnecting_gum_iTEDB-middle_D.2023_03_02_T.15_10_36_C.itedABCD_lab_2023.02.22_4.0v_8811_300s_CW150.root","READ"),
    "8811-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_AfterReconnecting_gum_iTEDB-middle_D.2023_03_02_T.15_10_36_C.itedABCD_lab_2023.02.22_4.0v_8811_300s_CW200.root","READ"),
    "8811-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_AfterReconnecting_gum_iTEDB-middle_D.2023_03_02_T.15_10_36_C.itedABCD_lab_2023.02.22_4.0v_8811_300s_CW250.root","READ"),
    "888-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_AfterReconnecting_gum_iTEDB-middle_D.2023_03_02_T.15_22_31_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100.root","READ"),
    "888-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_AfterReconnecting_gum_iTEDB-middle_D.2023_03_02_T.15_22_31_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW150.root","READ"),
    "888-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_AfterReconnecting_gum_iTEDB-middle_D.2023_03_02_T.15_22_31_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW200.root","READ"),
    "888-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137_AfterReconnecting_gum_iTEDB-middle_D.2023_03_02_T.15_22_31_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW250.root","READ")
}

iTEDC = {
    "8811-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDC-A1_D.2023_02_23_T.12_41_32_C.itedABCD_lab_2023.02.22_4.0v_8811_300s_CW100.root","READ"),
    "8811-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDC-A1_D.2023_02_23_T.12_41_32_C.itedABCD_lab_2023.02.22_4.0v_8811_300s_CW150.root","READ"),
    "8811-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDC-A1_D.2023_02_23_T.12_41_32_C.itedABCD_lab_2023.02.22_4.0v_8811_300s_CW200.root","READ"),
    "8811-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDC-A1_D.2023_02_23_T.12_41_32_C.itedABCD_lab_2023.02.22_4.0v_8811_300s_CW250.root","READ"),
    "888-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDC-A1_D.2023_02_23_T.13_26_03_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100.root","READ"),
    "888-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDC-A1_D.2023_02_23_T.13_26_03_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW150.root","READ"),
    "888-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDC-A1_D.2023_02_23_T.13_26_03_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW200.root","READ"),
    "888-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDC-A1_D.2023_02_23_T.13_26_03_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW250.root","READ")
}

iTEDD = {
    "8811-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDD-A1_D.2023_02_23_T.12_48_46_C.itedABCD_lab_2023.02.22_4.0v_8811_300s_CW100.root","READ"),
    "8811-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDD-A1_D.2023_02_23_T.12_48_46_C.itedABCD_lab_2023.02.22_4.0v_8811_300s_CW150.root","READ"),
    "8811-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDD-A1_D.2023_02_23_T.12_48_46_C.itedABCD_lab_2023.02.22_4.0v_8811_300s_CW200.root","READ"),
    "8811-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDD-A1_D.2023_02_23_T.12_48_46_C.itedABCD_lab_2023.02.22_4.0v_8811_300s_CW250.root","READ"),
    "888-100": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDD-A1_D.2023_02_23_T.13_03_18_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW100.root","READ"),
    "888-150": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDD-A1_D.2023_02_23_T.13_03_18_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW150.root","READ"),
    "888-200": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDD-A1_D.2023_02_23_T.13_03_18_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW200.root","READ"),
    "888-250": ROOT.TFile.Open("../../data/Multi_iTED_characterization/Cs137-iTEDD-A1_D.2023_02_23_T.13_03_18_C.itedABCD_lab_2023.02.22_4.0v_888_300s_CW250.root","READ")
}

In [7]:
for iTED in[0,1,2,3]:
    for Crystal in [0,1,2,3,4]:
        for Configuration in [888,8811]:
            for CW in [100,150,200,250]:
            
                spectra[
                    ["A","B","C","D"][iTED], Crystal
                ][
                    Configuration, CW
                ] = spectrum(
                    [iTEDA,iTEDB,iTEDC,iTEDD][iTED][f"{Configuration}-{CW}"].Get(
                        "{}_{}_amplitude_spectra;1".format(
                            "SCATTERER" if Crystal==0 else "ABSORBER",
                            ["A","B","C","D"][iTED] if Crystal==0 else "{}_{}".format(["A","B","C","D"][iTED],Crystal)
                        )
                    ), 
                    ["A","B","C","D"][iTED], 
                    Crystal, 
                    Configuration, 
                    CW, 
                    iTED_cal[:][["A","B","C","D"][iTED],Crystal]
                )

In [8]:
# Just verifying everything was loaded properly
spectra.dropna(inplace=True)
spectra.applymap(str)

iTED                             A                                            \
Crystal                          0             1             2             3   
Configuration Window                                                           
888           100      A.0.888.100   A.1.888.100   A.2.888.100   A.3.888.100   
              150      A.0.888.150   A.1.888.150   A.2.888.150   A.3.888.150   
              200      A.0.888.200   A.1.888.200   A.2.888.200   A.3.888.200   
              250      A.0.888.250   A.1.888.250   A.2.888.250   A.3.888.250   
8811          100     A.0.8811.100  A.1.8811.100  A.2.8811.100  A.3.8811.100   
              150     A.0.8811.150  A.1.8811.150  A.2.8811.150  A.3.8811.150   
              200     A.0.8811.200  A.1.8811.200  A.2.8811.200  A.3.8811.200   
              250     A.0.8811.250  A.1.8811.250  A.2.8811.250  A.3.8811.250   

iTED                                           B                              \
Crystal                          4             0             1             2   
Configuration Window                                                           
888           100      A.4.888.100   B.0.888.100   B.1.888.100   B.2.888.100   
              150      A.4.888.150   B.0.888.150   B.1.888.150   B.2.888.150   
              200      A.4.888.200   B.0.888.200   B.1.888.200   B.2.888.200   
              250      A.4.888.250   B.0.888.250   B.1.888.250   B.2.888.250   
8811          100     A.4.8811.100  B.0.8811.100  B.1.8811.100  B.2.8811.100   
              150     A.4.8811.150  B.0.8811.150  B.1.8811.150  B.2.8811.150   
              200     A.4.8811.200  B.0.8811.200  B.1.8811.200  B.2.8811.200   
              250     A.4.8811.250  B.0.8811.250  B.1.8811.250  B.2.8811.250   

iTED                                                         C                \
Crystal                          3             4             0             1   
Configuration Window                                                           
888           100      B.3.888.100   B.4.888.100   C.0.888.100   C.1.888.100   
              150      B.3.888.150   B.4.888.150   C.0.888.150   C.1.888.150   
              200      B.3.888.200   B.4.888.200   C.0.888.200   C.1.888.200   
              250      B.3.888.250   B.4.888.250   C.0.888.250   C.1.888.250   
8811          100     B.3.8811.100  B.4.8811.100  C.0.8811.100  C.1.8811.100   
              150     B.3.8811.150  B.4.8811.150  C.0.8811.150  C.1.8811.150   
              200     B.3.8811.200  B.4.8811.200  C.0.8811.200  C.1.8811.200   
              250     B.3.8811.250  B.4.8811.250  C.0.8811.250  C.1.8811.250   

iTED                                                                       D  \
Crystal                          2             3             4             0   
Configuration Window                                                           
888           100      C.2.888.100   C.3.888.100   C.4.888.100   D.0.888.100   
              150      C.2.888.150   C.3.888.150   C.4.888.150   D.0.888.150   
              200      C.2.888.200   C.3.888.200   C.4.888.200   D.0.888.200   
              250      C.2.888.250   C.3.888.250   C.4.888.250   D.0.888.250   
8811          100     C.2.8811.100  C.3.8811.100  C.4.8811.100  D.0.8811.100   
              150     C.2.8811.150  C.3.8811.150  C.4.8811.150  D.0.8811.150   
              200     C.2.8811.200  C.3.8811.200  C.4.8811.200  D.0.8811.200   
              250     C.2.8811.250  C.3.8811.250  C.4.8811.250  D.0.8811.250   

iTED                                                                          
Crystal                          1             2             3             4  
Configuration Window                                                          
888           100      D.1.888.100   D.2.888.100   D.3.888.100   D.4.888.100  
              150      D.1.888.150   D.2.888.150   D.3.888.150   D.4.888.150  
              200      D.1.888.200   D.2.888.200   D.3.888.200   D.4.888.200  
  

In [9]:
def get_resolution(cell):
    
    TH1D = cell.TH1D()
    
    TH1D.GetXaxis().SetRange(TH1D.FindBin(100),TH1D.FindBin(400))
    
    MaxBin   = TH1D.FindBin(TH1D.GetMaximumBin())
    
    ADC_Low  = MaxBin-60
    ADC_High = MaxBin+40

    gaussFit = ROOT.TF1("gaussFit", "pol1(0)+gaus(2)", ADC_Low, ADC_High)
    gaussFit.SetParameters(1000,-1,TH1D.GetMaximum(),MaxBin,10)
    TH1D.Fit(gaussFit,"QR")
    
    sigma = abs(gaussFit.GetParameter(4))
    centroid_ch = gaussFit.GetParameter(3)
    
    x1 = cell(centroid_ch+sigma*numpy.sqrt(2*numpy.log(2)))
    x2 = cell(centroid_ch-sigma*numpy.sqrt(2*numpy.log(2)))
    centroid = cell(centroid_ch)
        
    fwhm = x1-x2
            
    return sigma*numpy.sqrt(2*numpy.log(2))*2/centroid_ch*100, (fwhm/centroid)*100, centroid

In [10]:
def TH1D_draw(cell):
    
    TH1D = cell.TH1D()
    
    canvas = ROOT.TCanvas()
    canvas.cd()
    
    TH1D.SetTitle(repr(cell))
    TH1D.SetStats(False)
    
    latex = ROOT.TLatex()
    latex.SetNDC()
    latex.SetTextSize(0.03)
    
    TH1D.Draw("pe")
    
    l1,l2,l3 = get_resolution(cell)
    
    l4 = uproot.open(f"../../data/nTOF_March2022/{cell.Configuration()}/CW100ns/Resolutions_Cs137_CenterScatter_iTED{cell.iTED()}_8.8.{str(cell.Configuration())[2:]}_100ns.root:grResolEnergy;1").values()[1][cell.Crystal()]
    
    latex.DrawText(0.7, 0.8, "R_ch: {:.2f}%".format(l1))
    
    if (l2 > l4*1.1) and (l2 > 10):
        latex.DrawText(0.7, 0.75, "->R_E: {:.2f}%".format(l2))
    else:
        latex.DrawText(0.7, 0.75, "R_E: {:.2f}%".format(l2))
        
    if 662*(1-l2/100) < l3 < 662*(1+l2/100):
        latex.DrawText(0.7, 0.7, "E: {:.0f}keV".format(l3))
    else:
        latex.DrawText(0.7, 0.7, "->E: {:.0f}keV".format(l3))
        
    latex.DrawText(0.7, 0.65, "R_E(old): {:.2f}%".format(l4))
    latex.DrawText(0.7, 0.6, "Change: {:.2f}%".format(l2-l4))
            
    return canvas

### Energy resolution @662keV

#### Using channel values

In [11]:
spectra.applymap(lambda x: get_resolution(x)[0]).T.describe().drop(['count', 'std'], axis=0).style.background_gradient(cmap ='YlOrRd',axis=None)

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


#### Using the previous calibration (March 2022)

In [12]:
spectra.applymap(lambda x: get_resolution(x)[1]).T.describe().drop(['count', 'std'], axis=0).style.background_gradient(cmap ='YlOrRd',axis=None)

In [13]:
spectra.applymap(lambda x: get_resolution(x)[1]).style.background_gradient(cmap ='YlOrRd',axis=None)

### Energy of the Cesium peak using the previous calibration (March 2022)

In [14]:
spectra.applymap(lambda x: get_resolution(x)[2]).T.describe().drop(['count', 'std'], axis=0).style.background_gradient(cmap ='YlOrRd',axis=None)

In [15]:
canvas = spectra.applymap(TH1D_draw)
#canvas.applymap(lambda x: type(x).__name__)

In [16]:
for col in canvas.columns:
    for row in canvas.index:
        canvas.loc[row,col].Draw()

## Conclusions:

- The energy resolution of iTED-B-4 improved greatly.
- There doesn't seem to be any other major change.
- The next step should be doing the current analysis with the spectrum calibrated.